Data Challenge : Can you predict the tide ?
Author : Oumeima EL GHARBI
Date : November 2022

# PART 1 : Cleaning dataset

### Importing libraries

In [245]:
import numpy as np
import pandas as pd
%reset -f

from cleaning import *

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action="once")

%matplotlib inline
%autosave 300

Autosaving every 300 seconds


In [246]:
# Starting time
t0 = time()

### 1) Visualizing dataset

#### Loading dataset

In [247]:
#pd.set_option('display.float_format', lambda x: '%.2f' % x)
#pd.set_option('display.max_columns', None)

# load raw dataset
input_path = "./dataset/source/"

X_train_filename = "X_train_surge_new.npz"
Y_train_filename = "Y_train_surge.csv"
X_test_filename = "X_test_surge_new.npz"

X_train_file = "{}{}".format(input_path, X_train_filename)
Y_train_file = "{}{}".format(input_path, Y_train_filename)
X_test_file = "{}{}".format(input_path, X_test_filename)

In [248]:
X_train = np.load(X_train_file)
Y_train = pd.read_csv(Y_train_file)
X_test = np.load(X_test_file)

In [249]:
X_train

In [250]:
surge_train = np.array(Y_train)[:,1:]

In [251]:
surge_train

array([[ 0.58693592,  1.06958024,  0.76792754, ..., -0.42270688,
        -0.45623606, -0.82505705],
       [ 0.76792754, -0.1001619 ,  0.07077463, ..., -0.82505705,
        -0.99270295, -0.99270295],
       [ 0.07077463, -0.24428486, -0.35489084, ..., -0.99270295,
        -0.32211934, -0.88373311],
       ...,
       [ 0.48303332,  0.53330877,  1.44832196, ...,  1.22022298,
         1.44654495,  2.67036005],
       [ 1.44832196,  1.82036029,  1.66283054, ...,  2.67036005,
         2.62006628,  2.67874235],
       [ 1.66283054,  1.52876268,  1.28408882, ...,  2.67874235,
         1.22022298,  0.72566756]])

In [252]:
np.array(X_train)

array(['id_sequence', 't_slp', 'slp', 't_surge1_input', 'surge1_input',
       't_surge2_input', 'surge2_input', 't_surge1_output',
       't_surge2_output'], dtype='<U15')

In [253]:
pd.DataFrame(X_train)#.shape

,0
0,id_sequence
1,t_slp
2,slp
3,t_surge1_input
4,surge1_input
5,t_surge2_input
6,surge2_input
7,t_surge1_output
8,t_surge2_output


In [254]:
X_train['slp'].shape

(5599, 40, 41, 41)

In [255]:
nfields = 2; time_step_slp = 8
slp_train = []
slp_all = X_train['slp']
# ena :
n_slp = slp_all.shape[0]

for i in range(n_slp):
    slp_train.append(np.ndarray.flatten(slp_all[i,-1]))
    for j in range(1,nfields):
        slp_train[-1] = np.concatenate( ( slp_train[-1], np.ndarray.flatten(slp_all[i,-1-j*time_step_slp]) ) )
slp_train = np.array(slp_train)

slp_train

array([[100849.72, 100797.72, 100754.72, ..., 101970.81, 101981.81,
        101990.81],
       [100940.37, 100881.37, 100823.37, ..., 101515.72, 101562.72,
        101611.72],
       [101862.91, 101879.91, 101894.91, ..., 101005.37, 101038.37,
        101076.37],
       ...,
       [102048.13, 102080.13, 102106.13, ..., 100781.81, 100724.81,
        100664.81],
       [102111.95, 102134.95, 102149.95, ..., 101352.13, 101376.13,
        101392.13],
       [101972.53, 101984.53, 101993.53, ..., 100412.28, 100450.28,
        100483.28]], dtype=float32)

In [256]:
surge_train = np.array(Y_train)[:,1:]
surge_train

array([[ 0.58693592,  1.06958024,  0.76792754, ..., -0.42270688,
        -0.45623606, -0.82505705],
       [ 0.76792754, -0.1001619 ,  0.07077463, ..., -0.82505705,
        -0.99270295, -0.99270295],
       [ 0.07077463, -0.24428486, -0.35489084, ..., -0.99270295,
        -0.32211934, -0.88373311],
       ...,
       [ 0.48303332,  0.53330877,  1.44832196, ...,  1.22022298,
         1.44654495,  2.67036005],
       [ 1.44832196,  1.82036029,  1.66283054, ...,  2.67036005,
         2.62006628,  2.67874235],
       [ 1.66283054,  1.52876268,  1.28408882, ...,  2.67874235,
         1.22022298,  0.72566756]])

In [257]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
surge_train_df = pd.DataFrame(data=surge_train, columns=y_columns, index=X_train['id_sequence'])

surge_train_df

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
1,0.586936,1.069580,0.767928,-0.100162,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.053886,0.356847,0.348464,0.264641,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057
2,0.767928,-0.100162,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.348464,0.264641,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703
3,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733
4,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,-0.576103,1.160076,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733,-0.473001,-0.422707
5,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,-0.576103,1.160076,0.442813,0.305393,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733,-0.473001,-0.422707,-0.473001,-0.531677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,1.451674,1.220407,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,0.759197,0.884931,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700
5596,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223
5597,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.888589,1.545521,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223,1.446545,2.670360
5598,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.888589,1.545521,1.166779,0.653970,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223,1.446545,2.670360,2.620066,2.678742


In [258]:
from sklearn.model_selection import train_test_split

X_train_surge, X_test_surge, y_train_surge, y_test_surge = train_test_split(slp_train, surge_train_df, test_size=0.3, random_state=42)

In [259]:
slp_train.shape

(5599, 3362)

In [260]:
surge_train.shape

(5599, 20)

In [261]:
X_train_surge.shape

(3919, 3362)

In [262]:
y_test_surge.shape

(1680, 20)

In [263]:
X_train_surge

array([[102176.35, 102156.35, 102133.35, ..., 101640.01, 101663.01,
        101685.01],
       [101805.57, 101868.57, 101921.57, ..., 100350.2 , 100314.2 ,
        100281.2 ],
       [102114.1 , 102082.1 , 102049.1 , ..., 101215.24, 101180.24,
        101145.24],
       ...,
       [101786.12, 101687.12, 101576.12, ..., 100236.8 , 100318.8 ,
        100393.8 ],
       [102499.41, 102499.41, 102497.41, ..., 100472.49, 100492.49,
        100510.49],
       [101569.21, 101633.21, 101687.21, ...,  98750.56,  98741.56,
         98742.56]], dtype=float32)

In [264]:
y_test_surge

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
4441,-0.167196,-0.234230,-0.110217,-0.029776,0.047313,-0.046535,-0.354891,-0.405166,-0.314670,-0.690061,-0.104180,-0.162856,-0.104180,-0.548441,-0.783146,-0.481383,-0.774763,-0.422707,-0.296972,0.398758
2472,0.238359,2.105255,0.888589,1.062877,1.103097,-1.638591,-1.145891,-0.321374,-0.579455,-0.666599,-1.336377,-0.967556,-0.892115,-0.011974,-0.095797,0.105378,-0.095797,-0.162856,0.113760,0.331700
3317,-0.096810,-0.100162,-0.103514,0.265173,-0.244285,-0.321374,-0.552641,-0.579455,-0.646488,-0.110217,1.002283,0.960372,0.893313,0.943607,0.700521,0.457434,0.365229,0.608315,0.289788,0.390376
81,1.331013,0.553419,1.220407,0.637211,-0.013018,-0.150437,0.141160,0.198139,0.060720,-0.371649,-0.925645,-1.403435,-1.252554,-1.420200,-1.026232,-0.833439,-0.045504,-0.028739,-0.347266,-0.975938
4806,1.776788,3.053785,3.516319,2.557734,0.922106,2.068386,1.713106,1.153373,2.406907,1.947725,0.901696,1.236988,0.717285,0.650227,0.708903,0.356847,0.474199,0.222730,0.247877,0.055084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,-1.440841,-1.135836,-0.760446,-0.747039,-0.271098,-0.499014,-0.539234,-0.696764,-0.874404,-0.901217,1.295664,1.589044,1.228605,0.273024,-0.305355,-0.355649,-0.716087,-1.051379,-0.489765,-0.188003
4351,-0.116920,-0.328077,-0.371649,-0.455442,-0.455442,-0.499014,-0.143734,-0.009666,-0.096810,-0.485607,-0.364031,-0.322119,-0.489765,-0.791528,-0.556824,-0.690940,-0.422707,-0.674176,-0.196385,-0.313737
221,-1.420730,-1.561502,-1.702273,-1.826286,-1.779362,-1.283311,-1.249794,-0.921328,-0.991713,-0.777205,-0.380795,0.843020,-0.028739,-0.120944,-0.540059,-0.313737,-1.219025,-1.235790,-1.587846,-1.638140
1746,-0.163844,-0.458793,-0.006314,-0.090107,-0.250988,-0.368298,-0.214120,0.208194,0.074126,-0.056590,0.423905,0.524492,0.750814,0.532875,1.060959,0.641845,0.708903,0.775961,0.616698,0.474199


In [265]:
X = pd.DataFrame(X_train["id_sequence"])
X

,0
0,1
1,2
2,3
3,4
4,5
...,...
5594,5595
5595,5596
5596,5597
5597,5598


In [266]:
y = pd.DataFrame(Y_train)
y

,id_sequence,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,...,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
0,1,0.586936,1.069580,0.767928,-0.100162,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,...,-0.053886,0.356847,0.348464,0.264641,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057
1,2,0.767928,-0.100162,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,...,0.348464,0.264641,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703
2,3,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,...,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733
3,4,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,-0.576103,...,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733,-0.473001,-0.422707
4,5,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,-0.576103,1.160076,0.442813,...,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733,-0.473001,-0.422707,-0.473001,-0.531677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5594,5595,1.451674,1.220407,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,...,0.759197,0.884931,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700
5595,5596,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,...,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223
5596,5597,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.888589,...,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223,1.446545,2.670360
5597,5598,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.888589,1.545521,1.166779,...,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223,1.446545,2.670360,2.620066,2.678742


In [267]:
#X_train_surge, X_test_surge = train_test_split(slp_train, test_size=0.3, random_state=42)

In [268]:
#y_train_surge, y_test_surge = train_test_split(surge_train, test_size=0.3, random_state=42)

In [269]:
X_train_surge.shape

(3919, 3362)

In [270]:
y_train_surge.shape

(3919, 20)

In [271]:
#y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
#y_test_surge_save = pd.DataFrame(data=y_test_surge, columns=y_columns, index=X_test_surge['id_sequence'])

y_test_surge.to_csv("./dataset/output/Y_test_true.csv", index_label='id_sequence', sep=',')

### Predictive modeling

In [272]:
X_train_surge

array([[102176.35, 102156.35, 102133.35, ..., 101640.01, 101663.01,
        101685.01],
       [101805.57, 101868.57, 101921.57, ..., 100350.2 , 100314.2 ,
        100281.2 ],
       [102114.1 , 102082.1 , 102049.1 , ..., 101215.24, 101180.24,
        101145.24],
       ...,
       [101786.12, 101687.12, 101576.12, ..., 100236.8 , 100318.8 ,
        100393.8 ],
       [102499.41, 102499.41, 102497.41, ..., 100472.49, 100492.49,
        100510.49],
       [101569.21, 101633.21, 101687.21, ...,  98750.56,  98741.56,
         98742.56]], dtype=float32)

In [273]:
y_train_surge

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
5569,0.013796,0.000389,0.315448,0.107643,-0.086755,0.010444,0.271876,0.375779,0.281932,0.271876,0.432287,0.516110,0.105378,0.130525,0.440670,0.415523,0.138907,0.298171,0.122142,0.340082
2837,0.845017,0.556771,1.857229,0.372427,-0.914624,-0.304615,-0.411870,-0.750391,-0.626378,-0.559344,-1.302848,-1.042997,-1.084908,-0.783146,-0.992703,-1.571081,-1.403435,-1.017850,-0.313737,-0.598735
84,0.141160,0.198139,0.060720,-0.371649,0.265173,1.297496,1.860581,1.284089,1.274034,-0.133679,-0.045504,-0.028739,-0.347266,-0.975938,-0.908880,-1.219025,-1.260936,-1.068144,-0.732852,-0.196385
3659,-0.854294,-0.904569,-1.008472,-1.078857,-1.072154,-0.669950,-0.562696,-0.006314,0.245063,0.332207,-5.762229,-4.194740,-5.720317,-4.286945,-4.999440,-4.949146,-4.270180,-4.714442,-3.993565,-3.054748
791,-0.814073,-0.807370,-0.961548,-1.162650,-1.380510,-1.112374,-0.626378,-0.123624,0.214898,0.000389,-1.135202,-1.026232,-0.959174,-0.607117,-0.841822,-0.355649,-0.514912,-0.196385,-0.003592,-0.548441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773,1.072932,1.049470,0.741114,0.707597,0.848368,0.499792,0.335559,1.485191,1.565631,1.213703,0.465816,-0.540059,-0.674176,-0.975938,-0.489765,-0.372413,0.340082,0.599933,0.348464,0.314935
5192,-0.187306,0.070775,0.241711,0.402593,0.945567,1.448322,0.868478,0.781334,0.204842,-0.039831,0.155671,0.063466,0.390376,0.507728,0.306553,0.365229,0.432287,0.088613,0.080231,-0.179620
5227,-0.180603,-0.086755,0.754521,0.054016,0.975733,0.228304,0.224953,-0.190658,-0.328077,-0.485607,-0.070650,0.281406,-0.020357,0.197583,0.004790,0.490963,0.071849,0.843020,0.457434,0.558022
5391,0.395889,0.808148,0.590288,0.348965,0.077478,0.238359,0.188084,0.543364,1.062877,0.295338,0.901696,0.901696,0.331700,0.692138,-0.196385,-0.296972,-0.850204,-0.598735,-0.892115,-0.598735


#### 1) Ridge

In [274]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge


In [275]:
y_train_surge.values

array([[ 1.37957876e-02,  3.89000943e-04,  3.15448487e-01, ...,
         2.98170509e-01,  1.22142310e-01,  3.40081985e-01],
       [ 8.45016559e-01,  5.56770646e-01,  1.85722895e+00, ...,
        -1.01784984e+00, -3.13737041e-01, -5.98735077e-01],
       [ 1.41160261e-01,  1.98139104e-01,  6.07195408e-02, ...,
        -1.06814361e+00, -7.32851800e-01, -1.96384908e-01],
       ...,
       [-1.80602619e-01, -8.67551122e-02,  7.54520749e-01, ...,
         8.43019697e-01,  4.57434118e-01,  5.58021660e-01],
       [ 3.95889207e-01,  8.08147896e-01,  5.90287613e-01, ...,
        -5.98735077e-01, -8.92115409e-01, -5.98735077e-01],
       [ 2.72196669e+00,  2.67169124e+00,  2.09519941e+00, ...,
         4.07140347e-01,  2.64641328e-01,  9.01695763e-01]])

In [276]:
regr_ridge = MultiOutputRegressor(Ridge(random_state=123)).fit(X_train_surge, y_train_surge)


In [277]:
regr_ridge.predict(X_train_surge[[0]])

array([[ 0.0022316 , -0.14286804,  0.11506271, -0.57261276, -0.40707397,
         0.10398102,  0.49999428,  0.60402966,  0.46846724,  0.43014908,
         0.6303539 ,  0.8986492 ,  0.6192169 ,  0.4835968 ,  0.7785721 ,
         0.77897644,  0.49280548,  0.3391571 ,  0.41448975,  0.5651169 ]],
      dtype=float32)

In [278]:
y_test_surge_pred_ridge = regr_ridge.predict(X_test_surge)
y_test_surge_pred_ridge

array([[ 0.19655609,  0.12863922,  0.2917347 , ...,  0.63274384,
         0.17908478,  1.1499405 ],
       [ 2.1715126 ,  1.0897999 , -1.5926685 , ...,  2.5770912 ,
         2.4026527 ,  1.5600471 ],
       [ 1.1728554 ,  1.0215015 ,  0.20205307, ...,  0.66345596,
         0.6383686 ,  1.4873161 ],
       ...,
       [-0.59316254, -1.2878208 , -1.3967476 , ...,  1.8958855 ,
         1.6743107 ,  1.9851456 ],
       [ 0.6678734 ,  0.6780529 ,  1.1950817 , ..., -0.25061417,
         0.03788185,  0.8955536 ],
       [ 1.8836098 , -0.94682884, -0.6076412 , ..., -1.313076  ,
        -0.55200577,  0.2815628 ]], dtype=float32)

In [279]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
y_test_surge_pred_df_ridge = pd.DataFrame(data=y_test_surge_pred_ridge, columns=y_columns, index=y_test_surge.index)

y_test_surge_pred_df_ridge

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
4441,0.196556,0.128639,0.291735,-0.439983,-2.459488,-1.779236,-0.693396,-2.179199,-1.655435,-0.199747,0.410568,0.974117,-0.003826,-0.091866,0.337498,0.412544,0.056135,0.632744,0.179085,1.149940
2472,2.171513,1.089800,-1.592669,-2.470081,0.383476,1.465034,-0.310947,1.585719,0.483756,-0.826050,-2.505810,-2.610470,-2.843552,-0.573395,0.386337,0.839920,0.960018,2.577091,2.402653,1.560047
3317,1.172855,1.021502,0.202053,0.241264,-0.982498,-0.284241,-0.353062,-0.840775,0.058126,-0.177046,1.457232,0.781893,1.039833,1.174107,0.769917,0.235550,0.506546,0.663456,0.638369,1.487316
81,1.817589,2.019026,0.993914,1.459831,1.317787,0.863800,0.917665,0.281094,0.910346,2.258522,1.199459,0.652483,0.624050,0.276680,0.183144,-0.473152,-0.185604,0.023769,0.097548,0.567280
4806,1.116028,2.719082,1.704102,1.559845,1.064926,2.725563,2.252949,2.058954,-1.413400,-1.988955,1.966385,1.854366,3.251984,3.441391,3.196735,2.501152,2.224129,0.897083,-0.656326,-3.018978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,-0.464973,-1.131605,-2.678904,-3.450100,-1.650322,-0.415817,0.347073,0.987684,2.628694,-0.024794,4.040983,4.040279,3.682251,3.223328,1.042717,1.370262,1.403122,1.572796,1.486095,0.808834
4351,0.184101,0.062391,0.781113,0.543354,1.110390,-2.669464,-1.891502,-1.048905,-1.448190,-0.792767,-0.460892,0.029673,0.851696,0.739040,1.374779,1.142799,1.597317,3.536461,2.565464,1.891441
221,-0.593163,-1.287821,-1.396748,-2.327122,-2.369865,-4.128220,-2.235771,-2.986576,-3.286747,-3.927686,2.892214,1.555674,2.063347,1.347095,1.562855,0.230370,1.723152,1.895885,1.674311,1.985146
1746,0.667873,0.678053,1.195082,-0.178394,-0.975800,0.521553,0.404459,0.769419,1.076203,0.958242,0.763554,0.600649,0.610382,0.463791,0.298901,-0.023552,-0.452873,-0.250614,0.037882,0.895554


In [280]:
y_test_surge_pred_df_ridge.to_csv("./dataset/output/Y_test_pred_ridge.csv", index_label='id_sequence', sep=',')

In [281]:
y_test_surge

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
4441,-0.167196,-0.234230,-0.110217,-0.029776,0.047313,-0.046535,-0.354891,-0.405166,-0.314670,-0.690061,-0.104180,-0.162856,-0.104180,-0.548441,-0.783146,-0.481383,-0.774763,-0.422707,-0.296972,0.398758
2472,0.238359,2.105255,0.888589,1.062877,1.103097,-1.638591,-1.145891,-0.321374,-0.579455,-0.666599,-1.336377,-0.967556,-0.892115,-0.011974,-0.095797,0.105378,-0.095797,-0.162856,0.113760,0.331700
3317,-0.096810,-0.100162,-0.103514,0.265173,-0.244285,-0.321374,-0.552641,-0.579455,-0.646488,-0.110217,1.002283,0.960372,0.893313,0.943607,0.700521,0.457434,0.365229,0.608315,0.289788,0.390376
81,1.331013,0.553419,1.220407,0.637211,-0.013018,-0.150437,0.141160,0.198139,0.060720,-0.371649,-0.925645,-1.403435,-1.252554,-1.420200,-1.026232,-0.833439,-0.045504,-0.028739,-0.347266,-0.975938
4806,1.776788,3.053785,3.516319,2.557734,0.922106,2.068386,1.713106,1.153373,2.406907,1.947725,0.901696,1.236988,0.717285,0.650227,0.708903,0.356847,0.474199,0.222730,0.247877,0.055084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,-1.440841,-1.135836,-0.760446,-0.747039,-0.271098,-0.499014,-0.539234,-0.696764,-0.874404,-0.901217,1.295664,1.589044,1.228605,0.273024,-0.305355,-0.355649,-0.716087,-1.051379,-0.489765,-0.188003
4351,-0.116920,-0.328077,-0.371649,-0.455442,-0.455442,-0.499014,-0.143734,-0.009666,-0.096810,-0.485607,-0.364031,-0.322119,-0.489765,-0.791528,-0.556824,-0.690940,-0.422707,-0.674176,-0.196385,-0.313737
221,-1.420730,-1.561502,-1.702273,-1.826286,-1.779362,-1.283311,-1.249794,-0.921328,-0.991713,-0.777205,-0.380795,0.843020,-0.028739,-0.120944,-0.540059,-0.313737,-1.219025,-1.235790,-1.587846,-1.638140
1746,-0.163844,-0.458793,-0.006314,-0.090107,-0.250988,-0.368298,-0.214120,0.208194,0.074126,-0.056590,0.423905,0.524492,0.750814,0.532875,1.060959,0.641845,0.708903,0.775961,0.616698,0.474199


##### Evaluation

In [282]:
y_test_surge.values

array([[-0.16719583, -0.23422977, -0.11021699, ..., -0.42270688,
        -0.29697245,  0.39875805],
       [ 0.23835946,  2.1052545 ,  0.88858862, ..., -0.16285573,
         0.11376001,  0.33169969],
       [-0.0968102 , -0.1001619 , -0.1035136 , ...,  0.60831543,
         0.28978821,  0.39037576],
       ...,
       [-1.42073038, -1.56150164, -1.7022729 , ..., -1.23578951,
        -1.58784591, -1.63813968],
       [-0.16384414, -0.45879344, -0.00631439, ...,  0.77596134,
         0.61669773,  0.47419871],
       [ 0.07077463, -0.97160303, -0.87775552, ..., -1.06814361,
        -1.27770099, -0.91726229]])

#### 2) Ball Tree

In [283]:
from sklearn.neighbors import BallTree

tree = BallTree(X_train_surge, metric='l2')

In [284]:
X_test_surge.shape[0]

1680

In [285]:
surge_train

array([[ 0.58693592,  1.06958024,  0.76792754, ..., -0.42270688,
        -0.45623606, -0.82505705],
       [ 0.76792754, -0.1001619 ,  0.07077463, ..., -0.82505705,
        -0.99270295, -0.99270295],
       [ 0.07077463, -0.24428486, -0.35489084, ..., -0.99270295,
        -0.32211934, -0.88373311],
       ...,
       [ 0.48303332,  0.53330877,  1.44832196, ...,  1.22022298,
         1.44654495,  2.67036005],
       [ 1.44832196,  1.82036029,  1.66283054, ...,  2.67036005,
         2.62006628,  2.67874235],
       [ 1.66283054,  1.52876268,  1.28408882, ...,  2.67874235,
         1.22022298,  0.72566756]])

In [286]:
y_test_surge_pred_ballTree = []; k = 40

for i in range(X_test_surge.shape[0]):
    dist, ind = tree.query([X_test_surge[i]], k=k)
    y_test_surge_pred_ballTree.append(np.mean(surge_train[ind[0]], axis=0)) # we get indexes in surge_train (Y_train) where all the training data is


In [287]:
y_test_surge_pred_ballTree = np.array(y_test_surge_pred_ballTree)
y_test_surge_pred_ballTree

array([[-0.18202709, -0.0492999 ,  0.06381986, ...,  0.00919088,
         0.04733033,  0.00688575],
       [-0.17984849, -0.1620007 , -0.27503667, ..., -0.11046638,
        -0.00589725, -0.00401123],
       [-0.26992533, -0.08063827, -0.12027208, ..., -0.47907781,
        -0.39609309, -0.36130657],
       ...,
       [-0.22886705, -0.09630745,  0.07295323, ..., -0.21126348,
        -0.28397989, -0.2898475 ],
       [-0.29707408, -0.40466354, -0.39276502, ..., -0.15782635,
        -0.25275584, -0.2883806 ],
       [-0.24110074, -0.11809348, -0.06446633, ..., -0.45476916,
        -0.42186865, -0.32358624]])

In [288]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]

y_test_surge_pred_df_ballTree = pd.DataFrame(data=y_test_surge_pred_ballTree, columns=y_columns, index=y_test_surge.index)
y_test_surge_pred_df_ballTree.to_csv("./dataset/output/Y_test_pred_ballTree.csv", index_label='id_sequence', sep=',')


#### 3) XGBoost

In [290]:
from xgboost import XGBRegressor

In [291]:
regr_XGBoost = MultiOutputRegressor(XGBRegressor(random_state=42)).fit(X_train_surge, y_train_surge)

In [292]:
y_test_surge_pred_XGBoost = regr_XGBoost.predict(X_test_surge)
y_test_surge_pred_XGBoost

array([[-0.17394245, -0.07076971, -0.35313022, ..., -0.3765128 ,
        -0.05057035,  0.23735075],
       [ 0.61477417,  1.1349449 ,  1.5644692 , ...,  0.34753063,
         0.3756748 , -0.30959523],
       [ 0.20340942, -0.13879962,  0.00546361, ...,  0.19692692,
        -0.04792346,  0.0370005 ],
       ...,
       [-0.92727125, -1.1750612 , -1.4002796 , ...,  0.11028803,
         0.8300514 ,  0.48890892],
       [ 0.05323436, -0.01420574,  0.07312743, ...,  0.01672565,
        -0.26626652,  0.35633314],
       [ 0.25047988,  0.31369486, -0.03632465, ...,  0.4509365 ,
         0.10584661, -0.20588161]], dtype=float32)

In [293]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]

y_test_surge_pred_df_XGBoost = pd.DataFrame(data=y_test_surge_pred_XGBoost, columns=y_columns, index=y_test_surge.index)
y_test_surge_pred_df_XGBoost.to_csv("./dataset/output/Y_test_pred_XGBoost.csv", index_label='id_sequence', sep=',')

### Predict Y_test

In [294]:
slp_test = []
slp_all_test = X_test['slp']
for i in range(509):
    slp_test.append(np.ndarray.flatten(slp_all_test[i,-1]))
    for j in range(1,nfields):
        slp_test[-1] = np.concatenate( ( slp_test[-1], np.ndarray.flatten(slp_all_test[i,-1-j*time_step_slp]) ) )
slp_test = np.array(slp_test)



In [296]:
X_test['id_sequence']

array([5600, 5601, 5602, 5603, 5604, 5605, 5606, 5607, 5608, 5609, 5610,
       5611, 5612, 5613, 5614, 5615, 5616, 5617, 5618, 5619, 5620, 5621,
       5622, 5623, 5624, 5625, 5626, 5627, 5628, 5629, 5630, 5631, 5632,
       5633, 5634, 5635, 5636, 5637, 5638, 5639, 5640, 5641, 5642, 5643,
       5644, 5645, 5646, 5647, 5648, 5649, 5650, 5651, 5652, 5653, 5654,
       5655, 5656, 5657, 5658, 5659, 5660, 5661, 5662, 5663, 5664, 5665,
       5666, 5667, 5668, 5669, 5670, 5671, 5672, 5673, 5674, 5675, 5676,
       5677, 5678, 5679, 5680, 5681, 5682, 5683, 5684, 5685, 5686, 5687,
       5688, 5689, 5690, 5691, 5692, 5693, 5694, 5695, 5696, 5697, 5698,
       5699, 5700, 5701, 5702, 5703, 5704, 5705, 5706, 5707, 5708, 5709,
       5710, 5711, 5712, 5713, 5714, 5715, 5716, 5717, 5718, 5719, 5720,
       5721, 5722, 5723, 5724, 5725, 5726, 5727, 5728, 5729, 5730, 5731,
       5732, 5733, 5734, 5735, 5736, 5737, 5738, 5739, 5740, 5741, 5742,
       5743, 5744, 5745, 5746, 5747, 5748, 5749, 57

In [297]:
Y_test_pred = regr_XGBoost.predict(slp_test)
Y_test_pred

array([[-0.60546356, -0.35154304, -1.1281685 , ...,  0.3461988 ,
         0.14868052,  0.23383072],
       [-0.06356719, -0.13988079,  0.0609154 , ...,  0.84638095,
         0.45737076, -0.29219326],
       [-0.6290657 , -0.66719323, -0.28762132, ...,  0.04193096,
        -0.06487922, -0.33558333],
       ...,
       [-0.14261234, -0.50919014, -0.23794046, ..., -0.17575198,
         0.03501553, -0.06323517],
       [ 0.08064176,  0.45702848,  0.42545798, ..., -0.24848711,
         0.32236913,  0.36723155],
       [-0.33488104,  0.74959296,  1.951795  , ...,  0.22775625,
        -0.37415296,  0.07613017]], dtype=float32)

In [298]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]

Y_test_pred_df = pd.DataFrame(data=Y_test_pred, columns=y_columns, index=X_test['id_sequence'])
Y_test_pred_df.to_csv("./dataset/output/submit/Y_test.csv", index_label='id_sequence', sep=',')

### Saving final cleaned dataset

In [289]:
# End of pipeline time
t1 = time()
print("computing time : {:8.6f} sec".format(t1 - t0))
print("computing time : " + strftime('%H:%M:%S', gmtime(t1 - t0)))

computing time : 166.111978 sec
computing time : 00:02:46
